In [1]:
import numpy as np
from scipy.stats import unitary_group

np.set_printoptions(precision=3, suppress=True) #print less digits of floats for readability

In [2]:
from strawberryfields.decomposition_tests import Reck_decomposition,Reck_reconstruction

In [3]:
m=3
U = unitary_group.rvs(m)

In [4]:
U

array([[ 0.139+0.586j,  0.14 -0.255j,  0.739+0.079j],
       [ 0.082+0.219j, -0.888-0.152j, -0.11 +0.348j],
       [-0.657+0.388j, -0.224+0.23j , -0.002-0.561j]])

In [5]:
params_list = Reck_decomposition(U)

V matrix is unitary: [[ 1.+0.j -0.-0.j  0.-0.j]
 [ 0.+0.j  1.+0.j -0.-0.j]
 [ 0.+0.j -0.-0.j  1.+0.j]]


Reconstruction

In [6]:
Reck_reconstruction(m,params_list)

array([[ 0.139+0.586j,  0.14 -0.255j,  0.739+0.079j],
       [ 0.082+0.219j, -0.888-0.152j, -0.11 +0.348j],
       [-0.657+0.388j, -0.224+0.23j , -0.002-0.561j]])

Reconstruction of Reck Scheme in SF

In [7]:
import numpy as np
from strawberryfields.program import Program
from strawberryfields.ops import Rgate,sMZgate
from strawberryfields.engine import Engine


m=3

prog = Program(m)

with prog.context as q:
    
    for i,phi in enumerate(params_list['P_']):
        Rgate(phi) | q[i+1]

    for j in range(m-1):
        for k in range(len(params_list[j])):
            delta = params_list[j][k][0]
            sigma = params_list[j][k][1]
            theta1 = delta + sigma
            theta2 = sigma - delta
            sMZgate(theta1, theta2) | (q[j-k], q[j-k+1])
 
    for i,phi in enumerate(params_list['Q_']):
        Rgate(phi) | q[m-i-1]
        
eng = Engine("fock", backend_options={"cutoff_dim": 10})
result = eng.run(prog)
state = result.state

CircuitError: The operation sMZgate cannot be used with the compiler 'fock'.

# Clements

In [8]:
from strawberryfields.decomposition_tests import Clement_decomposition,Clement_reconstruction

In [14]:
m=6
U = unitary_group.rvs(m)

In [15]:
U

array([[-0.018-0.016j, -0.172-0.415j, -0.585-0.319j,  0.378-0.304j,
         0.022-0.118j,  0.273-0.172j],
       [ 0.483+0.518j,  0.486+0.047j, -0.141+0.218j,  0.049-0.11j ,
         0.382-0.028j,  0.051-0.169j],
       [-0.216+0.136j,  0.416-0.156j, -0.297+0.066j, -0.296+0.368j,
        -0.543-0.289j,  0.076-0.192j],
       [-0.15 -0.016j, -0.165-0.282j, -0.136-0.171j, -0.437+0.387j,
         0.564+0.17j , -0.174-0.325j],
       [ 0.065-0.175j,  0.283+0.205j, -0.251-0.337j,  0.101-0.159j,
        -0.131+0.169j, -0.755-0.121j],
       [-0.303-0.532j,  0.343+0.124j, -0.064+0.41j ,  0.372+0.113j,
         0.242+0.094j,  0.149-0.281j]])

In [16]:
params_list = Clement_decomposition(U)

V matrix is unitary: [[ 1.+0.j  0.+0.j -0.+0.j  0.-0.j  0.-0.j  0.+0.j]
 [-0.+0.j  1.+0.j -0.+0.j  0.+0.j -0.+0.j  0.+0.j]
 [-0.+0.j  0.-0.j  1.+0.j  0.+0.j  0.-0.j -0.+0.j]
 [ 0.+0.j  0.+0.j -0.-0.j  1.-0.j  0.+0.j  0.-0.j]
 [-0.+0.j -0.-0.j -0.+0.j  0.+0.j  1.-0.j  0.+0.j]
 [ 0.+0.j -0.+0.j -0.-0.j  0.+0.j -0.+0.j  1.-0.j]]


In [17]:
Clement_reconstruction(m,params_list)

array([[-0.018-0.016j, -0.172-0.415j, -0.585-0.319j,  0.378-0.304j,
         0.022-0.118j,  0.273-0.172j],
       [ 0.483+0.518j,  0.486+0.047j, -0.141+0.218j,  0.049-0.11j ,
         0.382-0.028j,  0.051-0.169j],
       [-0.216+0.136j,  0.416-0.156j, -0.297+0.066j, -0.296+0.368j,
        -0.543-0.289j,  0.076-0.192j],
       [-0.15 -0.016j, -0.165-0.282j, -0.136-0.171j, -0.437+0.387j,
         0.564+0.17j , -0.174-0.325j],
       [ 0.065-0.175j,  0.283+0.205j, -0.251-0.337j,  0.101-0.159j,
        -0.131+0.169j, -0.755-0.121j],
       [-0.303-0.532j,  0.343+0.124j, -0.064+0.41j ,  0.372+0.113j,
         0.242+0.094j,  0.149-0.281j]])